In [ ]:
import pandas as pd
import json
import datetime
import time
import os
import numpy as np
from config import DATA_MONTHLY_MEAN_NORMALIZED_FILE, DATA_CONSUMPTION_SEMI_PROCESSED_FILE, DATA_CONSUMPTION_PROCESSED_FILE, DATA_WEATHER_PROCESSED_FILE,  DATA_METADATA_PROCESSED_FILE, DATA_HOLIDAYS_PROCESSED_FILE, DATA_ISO_CONSUMPTION_PROCESSED_FILE, DATA_ENTHALPY_GRADIENTS_PROCESSED_FILE, DATA_SOLAR_GAINS_PROCESSED_FILE, DATA_GBM_CONSUMPTION_PROCESSED_FILE
from lgbm_imputer import imputer
import matplotlib
from sklearn.model_selection import GroupShuffleSplit,TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler

In [ ]:
df = pd.read_csv(DATA_CONSUMPTION_SEMI_PROCESSED_FILE)
df['timestamp'] = pd.to_datetime(df['timestamp'])
df = df.drop_duplicates(['timestamp', 'smapee'])
print(df.shape[0])

In [ ]:
#important so we can do the timeseries split for validation data
df['year'] = np.array(df['timestamp'].dt.year, dtype=np.uint16)
df['dayofweek'] = np.array(df['timestamp'].dt.dayofweek, dtype=np.uint8)
df['weekday'] = df['dayofweek'].apply(lambda x: 1 if 0<=x<5 else 0)
df.reset_index(inplace=True, drop=True)

In [ ]:
#get all in monthly data only for control group
ids = df['ID_CEA'].unique()
# for bid in ids:
df_final = df#[(df['INTERVENTION'].isin(['1CONTROL','2CONTROL', '3CONTROL']))]
df_final =  df_final.set_index('timestamp').resample('M').mean()
df_final['CONSUMPTION_kWh'].plot()
df_final

In [ ]:
minimum =  0.50
scaler = MinMaxScaler(feature_range=(minimum, 1))
df_final['consumption_kWh_norm'] = scaler.fit_transform(df_final['CONSUMPTION_kWh'].values.reshape(-1, 1))
df_final

In [ ]:
df_final['consumption_kWh_norm'].plot()

In [ ]:
df_final.T

In [ ]:
# df_final['consumption_kWh_norm_2019'] = list(df_final['consumption_kWh_norm'][12:].values) + list(range(12))
df_final.T.to_csv(DATA_MONTHLY_MEAN_NORMALIZED_FILE)